In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Load the Adult dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
column_names = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 
                'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']
df = pd.read_csv(url, names=column_names, skipinitialspace=True, na_values='?')

# Assuming X is feature DataFrame and y is target
label_encoders = {}
for column in X.columns:
    if X[column].dtype == 'object':
        le = LabelEncoder()
        X[column] = le.fit_transform(X[column])
        label_encoders[column] = le

# Preprocess the data
df = df.dropna()
df['income'] = df['income'].map({'>50K': 1, '<=50K': 0})
sensitive_attribute = 'sex'
label = 'income'

# Split the data
X = df.drop('income', axis=1)
y = df['income']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

NameError: name 'X' is not defined

In [2]:
!pip install tensorflow
!pip install fairlearn
!pip install inFairness


  Using cached tensorflow-2.17.0-cp39-cp39-macosx_12_0_arm64.whl.metadata (4.1 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached h5py-3.11.0-cp39-cp39-macosx_11_0_arm64.whl.metadata (2.5 kB)
  Using cached libclang-18.1.1-1-py2.py3-none-macosx_11_0_arm64.whl.metadata (5.2 kB)
  Using cached ml_dtypes-0.4.0-cp39-cp39-macosx_10_9_universal2.whl.metadata (20 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached protobuf-4.25.4-cp37-abi3-macosx_10_9_universal2.whl.metadata (541 bytes)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached wrapt-1.16.0-cp39-cp39-macosx_11_0_arm64.whl.metadata (6.6 kB)
  Using cached grpcio-1.

In [2]:
from sklearn.preprocessing import LabelEncoder

# Create a LabelEncoder object
le = LabelEncoder()

# Apply LabelEncoder to each categorical column
for column in X_train.select_dtypes(include=['object']).columns:
    X_train[column] = le.fit_transform(X_train[column])


NameError: name 'X_train' is not defined

In [22]:
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric

# Assuming `income` is your label and `sensitive_attribute` is defined
train_data = BinaryLabelDataset(df=pd.concat([X_train, y_train], axis=1), 
                                label_names=['income'], 
                                protected_attribute_names=[sensitive_attribute])

# Measure bias
metric = BinaryLabelDatasetMetric(train_data, unprivileged_groups=[{sensitive_attribute: 0}], 
                                  privileged_groups=[{sensitive_attribute: 1}])
print(f"Disparate impact before mitigation: {metric.disparate_impact()}")


Disparate impact before mitigation: 0.3647382591137069


In [6]:
from aif360.algorithms.preprocessing import Reweighing

# Apply bias mitigation technique (Reweighing)
rw = Reweighing(unprivileged_groups=[{sensitive_attribute: 0}], 
                privileged_groups=[{sensitive_attribute: 1}])
train_data_transformed = rw.fit_transform(train_data)

# Measure bias after mitigation
metric_transformed = BinaryLabelDatasetMetric(train_data_transformed, 
                                              unprivileged_groups=[{sensitive_attribute: 0}], 
                                              privileged_groups=[{sensitive_attribute: 1}])
print(f"Disparate impact after mitigation: {metric_transformed.disparate_impact()}")

Disparate impact after mitigation: 0.9999999999999998


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

# X and y are features and target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Identify categorical columns in your dataset
categorical_columns = X.select_dtypes(include=['object']).columns
numeric_columns = X.select_dtypes(include=['int64', 'float64']).columns

# Create a preprocessing pipeline to handle categorical and numeric data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_columns),
        ('cat', OneHotEncoder(drop='first'), categorical_columns)
    ])

# Define a pipeline with preprocessing and the Logistic Regression model
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=5000))
])

def train_and_evaluate(X_train, X_test, y_train, y_test):
    # Train the model
    model_pipeline.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model_pipeline.predict(X_test)
    
    # Print the classification report
    print(classification_report(y_test, y_pred))

# Original data
print("Model performance on original data:")
train_and_evaluate(X_train, X_test, y_train, y_test)

# If you have transformed data, make sure it's split into train and test sets
if 'train_data_transformed' in locals():
    X_train_transformed = train_data_transformed.features
    y_train_transformed = train_data_transformed.labels.ravel()
    
    # Convert numpy array to pandas DataFrame
    if isinstance(X_train_transformed, np.ndarray):
        X

Model performance on original data:
              precision    recall  f1-score   support

           0       0.88      0.93      0.90      4503
           1       0.76      0.62      0.68      1530

    accuracy                           0.85      6033
   macro avg       0.82      0.78      0.79      6033
weighted avg       0.85      0.85      0.85      6033



In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

def prepare_data(X):
    # Identify categorical and numeric columns
    categorical_columns = X.select_dtypes(include=['object']).columns
    numeric_columns = X.select_dtypes(include=['int64', 'float64']).columns

    # Create preprocessing pipeline
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numeric_columns),
            ('cat', OneHotEncoder(drop='first'), categorical_columns)
        ])

    # Create model pipeline
    model_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', LogisticRegression(max_iter=5000))
    ])
    
    return model_pipeline

def train_and_evaluate(X_train, X_test, y_train, y_test, model_name="Model"):
    # Prepare the data and model
    model = prepare_data(X_train)
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    
    # Print results
    print(f"\n{model_name} Performance:")
    print(f"Accuracy: {accuracy:.4f}")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))
    
    return model, accuracy

# Preprocess the data
def preprocess_data(X):
    categorical_columns = X.select_dtypes(include=['object']).columns
    for col in categorical_columns:
        X[col] = X[col].astype('category').cat.codes
    return X

# Preprocess original data
X_train_preprocessed = preprocess_data(X_train)
X_test_preprocessed = preprocess_data(X_test)

# Evaluate on original data
print("Evaluating on original data:")
original_model, original_accuracy = train_and_evaluate(X_train_preprocessed, X_test_preprocessed, y_train, y_test, "Original Model")

# Apply bias mitigation to preprocessed data
train_data_preprocessed = BinaryLabelDataset(df=pd.concat([X_train_preprocessed, y_train], axis=1), 
                                             label_names=['income'], 
                                             protected_attribute_names=[sensitive_attribute])

rw = Reweighing(unprivileged_groups=[{sensitive_attribute: 0}], 
                privileged_groups=[{sensitive_attribute: 1}])
train_data_transformed = rw.fit_transform(train_data_preprocessed)

# Transform test data
test_data_preprocessed = BinaryLabelDataset(df=pd.concat([X_test_preprocessed, y_test], axis=1), 
                                            label_names=['income'], 
                                            protected_attribute_names=[sensitive_attribute])
test_data_transformed = rw.transform(test_data_preprocessed)

# Prepare transformed data for evaluation
X_train_transformed = pd.DataFrame(train_data_transformed.features, columns=X_train.columns)
y_train_transformed = train_data_transformed.labels.ravel()
X_test_transformed = pd.DataFrame(test_data_transformed.features, columns=X_test.columns)
y_test_transformed = test_data_transformed.labels.ravel()

# Evaluate on transformed data
print("\nEvaluating on transformed (mitigated) data:")
transformed_model, transformed_accuracy = train_and_evaluate(X_train_transformed, X_test_transformed, 
                                                             y_train_transformed, y_test_transformed, 
                                                             "Transformed Model")

# Compare accuracies
print("\nAccuracy Comparison:")
print(f"Original Model Accuracy: {original_accuracy:.4f}")
print(f"Transformed Model Accuracy: {transformed_accuracy:.4f}")
print(f"Accuracy Difference (Transformed - Original): {transformed_accuracy - original_accuracy:.4f}")

Evaluating on original data:

Original Model Performance:
Accuracy: 0.8122

Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.95      0.88      4503
           1       0.73      0.41      0.52      1530

    accuracy                           0.81      6033
   macro avg       0.78      0.68      0.70      6033
weighted avg       0.80      0.81      0.79      6033


Evaluating on transformed (mitigated) data:

Transformed Model Performance:
Accuracy: 0.8230

Classification Report:
              precision    recall  f1-score   support

         0.0       0.84      0.95      0.89      4503
         1.0       0.74      0.46      0.57      1530

    accuracy                           0.82      6033
   macro avg       0.79      0.70      0.73      6033
weighted avg       0.81      0.82      0.81      6033


Accuracy Comparison:
Original Model Accuracy: 0.8122
Transformed Model Accuracy: 0.8230
Accuracy Difference (Transformed - Origi